# End to End Chicken Disease Classification

## Stage 4: Model Evaluation with mlflow

In [1]:
%pwd

'd:\\Documents\\Projects\\chiicken_disease_classification\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'd:\\Documents\\Projects\\chiicken_disease_classification'

### Collect mlflow url 

MLFLOW_TRACKING_URI=https://dagshub.com/amulyaprasanth/chiicken_disease_classification.mlflow \
MLFLOW_TRACKING_USERNAME=amulyaprasanth \
MLFLOW_TRACKING_PASSWORD=d6817a3e2dd02437d35d18934f5ec3e3d99cfd35 \
python script.py

### Update entity


In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    trained_model_path: Path
    training_data: Path
    all_params: dict
    mlflow_url: str
    params_batch_size: int
    params_image_size: list

### Update configuration manager in src config 

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILEPATH,
                 params_filepath=PARAMS_FILEPATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        evaluation_config = EvaluationConfig(
            trained_model_path=Path(self.config.training.trained_model_path),
            training_data=os.path.join(
                self.config.data_ingestion.unzip_dir, "Chicken-fecal-images"),
            mlflow_url="https://dagshub.com/amulyaprasanth/chiicken_disease_classification.mlflow ",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return evaluation_config

### Update components

In [6]:
import shutil
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def prepare_val_data(self):
        val_ds = tf.keras.utils.image_dataset_from_directory(
            directory=self.config.training_data,
            validation_split=0.2,
            subset="validation",
            seed=42,
            batch_size=self.config.params_batch_size,
            image_size=(self.config.params_image_size[0], self.config.params_image_size[1]))

        self.val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

    def evaluate(self):
        self.model = self.load_model(self.config.trained_model_path)
        self.prepare_val_data()
        self.score = self.model.evaluate(self.val_ds)
        self.save_score()

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_url)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model,
                                       "model",
                                       registered_model_name="VGG16Model")

            else:
                mlflow.keras.log_model(self.model, "model")
    def copy_model_for_deployment(self):
        '''
        This function copies the trained model file to  model folder to be used for deployment
        '''
        os.makedirs("./models", exist_ok=True)
        shutil.copy(
            src = self.config.trained_model_path,
            dst = Path("./models"))
        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

### Update pipeline

In [7]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    evaluation = Evaluation(config=evaluation_config)
    evaluation.evaluate()
    evaluation.copy_model_for_deployment()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-10-19 22:28:16,427:  INFO: common: yaml file: config\config.yaml loaded]
[2023-10-19 22:28:16,430:  INFO: common: yaml file: params.yaml loaded]
[2023-10-19 22:28:16,431:  INFO: common: Created directory at artifacts]
Found 390 files belonging to 2 classes.
Using 78 files for validation.
5/5 [==============================] - 9s 659ms/step - loss: 0.4801 - accuracy: 0.4872
[2023-10-19 22:28:27,272:  INFO: common: json saved at scores.json]


2023/10/19 22:28:27 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2023-10-19 22:28:29,134:  WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 13). These functions will not be directly callable after loading.]
[2023-10-19 22:28:30,091:  INFO: builder_impl: Assets written to: C:\Users\amuly\AppData\Local\Temp\tmptx2lnwva\model\data\model\assets]
[2023-10-19 22:28:40,827:  INFO: dir_util: creating D:\Documents\Projects\chiicken_disease_classification\mlruns\0\c9b71c56abe14b83bb8473f7fa63bd68\artifacts\model\data]
[2023-10-19 22:28:40,830:  INFO: dir_util: creating D:\Documents\Projects\chiicken_disease_classification\mlruns\0\c9b71c56abe14b83bb8473f7fa63bd68\artifacts\model\data\model]
[2023-10-19 22:28:40,832:  INFO: dir_util: creating D:\Documents\Projects\chiicken_disease_classification\mlruns\0\c9b71c56abe14b83bb8473f7fa63bd68\artifacts\model\data\model\assets]
[2023-10-19 22:2

d:\Documents\Projects\chiicken_disease_classification\venv\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
